In [ ]:
from PIL import Image
from IPython.display import display
import tkinter as tk
from tkinter import filedialog
import hashlib 
import binascii
import textwrap
import cv2
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from importlib import reload  
from bisect import bisect_left as bsearch
import qrng
import math
from skimage import io
%matplotlib inline

In [ ]:
def generate_rn():
    qrng.set_provider_as_IBMQ('')
    qrng.set_backend()
    return qrng.get_random_float(0,1)

In [ ]:
E1=[]
for i in range(65536*3):
    x=generate_rn()
    E1.append(x)

E1=np.array(E1)

In [ ]:
E1=E1.reshape(-1)

In [ ]:
class Layer_Dense:
    
    def __init__(self,n_inputs,n_neurons):
        np.random.seed(0)
        self.n_input=n_inputs
        self.n_neurons=n_neurons
        self.create_weights()
        self.create_biases()
    
    def create_weights(self):
        self.weights=0.1*np.random.rand(self.n_input,self.n_neurons)
        
        
    def create_biases(self):
        self.biases=np.zeros((1,self.n_neurons))
        
    def forward(self,inputs):
        self.output=np.dot(inputs,self.weights)+self.biases
        

In [ ]:
class Activation_sigmoid:
    def forward(self,inputs):
        self.output=1/(1 + np.exp(-inputs))

In [ ]:
X=[]
X=inputKey(X)
X=np.array(X)

In [ ]:
Hidden_layer1=Layer_Dense(64,32)
Activation1=Activation_sigmoid()
Output_layer=Layer_Dense(32,3) 
Activation2=Activation_sigmoid()

In [ ]:
Hidden_layer1.forward(X)
Activation1.forward(Hidden_layer1.output)
Output_layer.forward(Activation1.output)
Activation2.forward(Output_layer.output)
Enc=[]
Enc=Activation2.output
Enc=np.array(Enc)

In [ ]:
E2=[]
for i in range(65536):
    x=generate_rn()
    E2.append(x)

E2=np.array(E2)

In [ ]:
E3=[]
for i in range(65536):
    x=generate_rn()
    E3.append(x)

E3=np.array(E3)

In [ ]:
E4=[]
for i in range(65536):
    x=generate_rn()
    E4.append(x)

E4=np.array(E4)

In [ ]:
E2=E2.reshape(-1)
E3=E3.reshape(-1)
E4=E4.reshape(-1)

In [ ]:
def image_selector():                           
    path = "NULL"
    root = tk.Tk()
    root.withdraw()                             
    path = filedialog.askopenfilename()         
    if path!="NULL":
        print("Image loaded!") 
    else:
        print("Error Image not loaded!")
    return path

In [ ]:
def split_into_rgb_channels(image):
    b,g,r = cv2.split(image)
    return b, g, r

In [ ]:
def decompose_matrix(iname):
    image = cv2.imread(iname)
    blue,green,red = split_into_rgb_channels(image)
    for values, channel in zip((red, green, blue), (2,1,0)):
        img = np.zeros((values.shape[0], values.shape[1]), dtype = np.uint8)
        img[:,:] = (values)
        if channel == 0:
            B = np.asmatrix(img)
        elif channel == 1:
            G = np.asmatrix(img)
        else:
            R = np.asmatrix(img)
    return B,G,R

In [ ]:
def pixel_permutation(iname,E1):
    IrgbH=iname.reshape(-1)
    indx = [i[0] for i in sorted(enumerate(E1), key=lambda x:x[1])]
    IrgbH_new = IrgbH[np.ix_(indx)]
    IrgbH_new= np.reshape(IrgbH_new, (256, 256))
    return IrgbH_new

In [ ]:
def permute(b,g,r,E1):
    blue=pixel_permutation(b,E1)
    green=pixel_permutation(g,E1)
    red=pixel_permutation(r,E1)
    return blue,green,red

In [ ]:
def normalize(X,mod,c):
    m=X.shape
    return ((np.round(X*pow(10,4)))%mod) + c*np.ones(m) 

In [ ]:
def key_generation(X):
    E2_n=normalize(X,256,0)
    E2_n=np.array(E2_n)
    E2_n =E2_n.astype('int32')
    #E2_n = np.reshape(E2_n, (256, 256))
    E2_n=E2_n.reshape(-1)
    return E2_n

In [ ]:
def Dna_encoding(A,num):
    t=len(A)
    fv=np.zeros((t,),dtype = str)
    for k in range(t):
        if num[k]==1 or num[k]==2:
            if A[k]==0:
                fv[k]='A'
            elif A[k]==3:
                fv[k]='T'
            elif A[k]==2:
                if num[k]==1:
                    fv[k]='C'
                else:
                    fv[k]='G'
            else:
                if num[k]==1:
                    fv[k]='G'
                else:
                    fv[k]='C'

        elif num[k]==3 or num[k]==4:
            if A[k]==1:
                fv[k]='A'
            elif A[k]==2:
                fv[k]='T'
            elif A[k]==0:
                if num[k]==3:
                    fv[k]='G'
                else:
                    fv[k]='C'
            else:
                if num[k]==3:
                    fv[k]='C'
                else:
                    fv[k]='G'

        elif num[k]==5 or num[k]==6:
            if A[k]==2:
                fv[k]='A'
            elif A[k]==1:
                fv[k]='T'
            elif A[k]==0:
                if num[k]==5:
                    fv[k]='G'
                else:
                    fv[k]='C'
            else:
                if num[k]==5:
                    fv[k]='C'
                else:
                    fv[k]='G'

        else:                    #  num==7 || num==8
            if A[k]==3:
                fv[k]='A'
            elif A[k]==0:
                fv[k]='T'
            elif A[k]==1:
                if num[k]==7:
                    fv[k]='G'
                else:
                    fv[k]='C'
            else:
                if num[k]==7:
                    fv[k]='C'
                else:
                    fv[k]='G'
    return fv

In [ ]:
def pair_dna(A,num):
    a1=np.bitwise_and(A,192)/64
    a2=np.bitwise_and(A,48)/16
    a3=np.bitwise_and(A,12)/4
    a4=np.bitwise_and(A,3)
    a1=a1.reshape(-1)
    a2=a2.reshape(-1)
    a3=a3.reshape(-1)
    a4=a4.reshape(-1)
    a1_dna=Dna_encoding(a1,num)
    a2_dna=Dna_encoding(a2,num)
    a3_dna=Dna_encoding(a3,num)
    a4_dna=Dna_encoding(a4,num)
    X=np.array([a1_dna,a2_dna,a3_dna,a4_dna])
    return X

In [ ]:
def encode(blue,green,red,key,E1,E2):
    E_1=normalize(E1,8,1)
    E_2=normalize(E2,8,1)
    blue_dna=pair_dna(blue,E_1)
    green_dna=pair_dna(green,E_1)
    red_dna=pair_dna(red,E_1)
    key_dna=pair_dna(key,E_2)
    return blue_dna,green_dna,red_dna,key_dna

In [ ]:
#Subfunctions DNA operations arr1 and arr2 represent two DNA matrices to be operated.
#When num is 0, it means to add; when num is 1, it means to subtract; when num is 2, it means XOR; when num is 3, it means XOR
def DNA_operation(arr1,arr2,num):
    t=len(arr1)
    fv=np.zeros((t,),dtype = str)        #Pre-allocated memory
    for i in range(t):
        if num[i]==0:                #add
            if arr1[i]=='A':
                fv[i]=arr2[i]
            elif arr1[i]=='T':
                if arr2[i]=='A':
                    fv[i]='T'
                elif arr2[i]=='T':
                    fv[i]='C'
                elif arr2[i]=='C':
                    fv[i]='G'
                else:
                    fv[i]='A'
            elif arr1[i]=='C':
                if arr2[i]=='A':
                    fv[i]='C'
                elif arr2[i]=='T':
                    fv[i]='G'
                elif arr2[i]=='C':
                    fv[i]='A'
                else:
                    fv[i]='T'
            else:
                if arr2[i]=='A':
                    fv[i]='G'
                elif arr2[i]=='T':
                    fv[i]='A'
                elif arr2[i]=='C':
                    fv[i]='T'
                else:
                    fv[i]='C'

        elif num[i]==1:                  #Subtraction
            if arr2[i]=='A':
                fv[i]=arr1[i]
            elif arr2[i]=='T':
                if arr1[i]=='A':
                    fv[i]='G'
                elif arr1[i]=='T':
                    fv[i]='A'
                elif arr1[i]=='C':
                    fv[i]='T'
                else:
                    fv[i]='C'
            elif arr2[i]=='C':
                if arr1[i]=='A':
                    fv[i]='C'
                elif arr1[i]=='T':
                    fv[i]='G'
                elif arr1[i]=='C':
                    fv[i]='A'
                else:
                    fv[i]='T'
            else:
                if arr1[i]=='A':
                    fv[i]='T'
                elif arr1[i]=='T':
                    fv[i]='C';
                elif arr1[i]=='C':
                    fv[i]='G'
                else:
                    fv[i]='A'
        elif num[i]==2:        #XOR
            if arr1[i]==arr2[i]:
                fv[i]='C'
            elif (arr1[i]=='T' and arr2[i]=='A') or (arr1[i]=='A' and arr2[i]=='T') or (arr1[i]=='G' and arr2[i]=='C') or (arr1[i]=='C' and arr2[i]=='G'):
                fv[i]='G'
            elif (arr1[i]=='C' and arr2[i]=='A') or (arr1[i]=='A' and arr2[i]=='C') or (arr1[i]=='G' and arr2[i]=='T') or (arr1[i]=='T' and arr2[i]=='G'):
                fv[i]='A'
            else:
                fv[i]='T'

        else:                #%same xor
            if arr1[i]==arr2[i]:
                fv[i]='A'
            elif (arr1[i]=='T' and arr2[i]=='A') or (arr1[i]=='A' and arr2[i]=='T') or (arr1[i]=='G' and arr2[i]=='C') or (arr1[i]=='C' and arr2[i]=='G'):
                fv[i]='T'
            elif (arr1[i]=='C' and arr2[i]=='A') or (arr1[i]=='A' and arr2[i]=='C') or (arr1[i]=='G' and arr2[i]=='T') or (arr1[i]=='T' and arr2[i]=='G'):
                fv[i]='C'
            else:
                fv[i]='G'
    
    return fv


In [ ]:
def pair_operation(X,Y,Z):
    I1=DNA_operation(X[0,:],Y[0,:],Z)
    I2=DNA_operation(X[1,:],Y[1,:],Z)
    I3=DNA_operation(X[2,:],Y[2,:],Z)
    I4=DNA_operation(X[3,:],Y[3,:],Z)
    A=np.array([I1,I2,I3,I4])
    
    return A

In [ ]:
def operation(blue,green,red,key,E3):
    E_3=normalize(E3,4,0)
    I_Bo=pair_operation(blue,key,E_3)
    I_Go=pair_operation(green,key,E_3)
    I_Ro=pair_operation(red,key,E_3)
    return I_Bo,I_Go,I_Ro

In [ ]:
# Sub-function DNA decoding
def DNA_decoding(array,num):
    t=len(array)
    A=np.zeros((t,))        #Pre-allocated memory
   
    for i in range(t):
        if num[i]==1 or num[i]==2:
            if array[i]=='A':
                A[i]=0
            elif array[i]=='T':
                A[i]=3
            elif array[i]=='G':
                if num[i]==1:
                    A[i]=1
                else:
                    A[i]=2
            else:
                if num[i]==1:
                    A[i]=2
                else:
                    A[i]=1
        elif num[i]==3 or num[i]==4:
            if array[i]=='A':
                A[i]=1
            elif array[i]=='T':
                A[i]=2
            elif array[i]=='G':
                if num[i]==3:
                    A[i]=0
                else:
                    A[i]=3
            else:
                if num[i]==3:
                    A[i]=3
                else:
                    A[i]=0
        elif num[i]==5 or num[i]==6:
            if array[i]=='A':
                A[i]=2
            elif array[i]=='T':
                A[i]=1
            elif array[i]=='G':
                if num[i]==5:
                    A[i]=0
                else:
                    A[i]=3
            else:
                if num[i]==5:
                    A[i]=3
                else:
                    A[i]=0
        else:                    #%  num==7 || num==8
            if array[i]=='A':
                A[i]=3
            elif array[i]=='T':
                A[i]=0;
            elif array[i]=='G':
                if num[i]==7:
                    A[i]=1
                else:
                    A[i]=2
            else:
                if num[i]==7:
                    A[i]=2
                else:
                    A[i]=1
                    
    return A

In [ ]:
def pair_decoding(X,Z):
    I1=DNA_decoding(X[0,:],Z)
    I2=DNA_decoding(X[1,:],Z)
    I3=DNA_decoding(X[2,:],Z)
    I4=DNA_decoding(X[3,:],Z)
    A=I1*64+I2*16+I3*4+I4
    return A

In [ ]:
def decode(blue,green,red,E4):
    E_4=normalize(E4,8,1)
    I_Bd=pair_decoding(blue,E_4)
    I_Gd=pair_decoding(green,E_4)
    I_Rd=pair_decoding(red,E_4)
    return I_Bd,I_Gd,I_Rd

In [ ]:
def diffusion(I1,I2,I3,E):
    m,n=I1.shape
    for i in range(m):
        for j in range(1,n):
            I1[i][j]=I1[i][j-1]^I2[i][j]^I3[i][j]^E[i][j]
    
    return I1

In [ ]:
def diffuse(blue,green,red,E2,E3,E4):
    E2_n=normalize(E2,256,0)
    E3_n=normalize(E3,256,0)
    E4_n=normalize(E4,256,0)
    I_Bd=blue.astype(int)
    I_Gd=green.astype(int)
    I_Rd=red.astype(int)
    E2_n=E2_n.astype(int)
    E3_n=E3_n.astype(int)
    E4_n=E4_n.astype(int)
    I_bpd = np.bitwise_xor(I_Bd, E4_n)
    I_gpd = np.bitwise_xor(I_Gd, E3_n)
    I_rpd = np.bitwise_xor(I_Rd, E2_n)
    I_rpd=np.reshape(I_rpd, (256, 256))
    I_gpd=np.reshape(I_gpd, (256, 256))
    I_bpd=np.reshape(I_bpd, (256, 256))
    E2_n=np.reshape(E2_n, (256, 256))
    E3_n=np.reshape(E3_n, (256, 256))
    E4_n=np.reshape(E4_n, (256, 256))
    I_Rd=np.reshape(I_Rd, (256, 256))
    I_Gd=np.reshape(I_Gd, (256, 256))
    I_Bd=np.reshape(I_Bd, (256, 256))
    I_Rdi=diffusion(I_rpd,I_Rd,I_Gd,E2_n)
    I_Gdi=diffusion(I_gpd,I_Gd,I_Bd,E3_n)
    I_Bdi=diffusion(I_bpd,I_Bd,I_Rd,E4_n)
    return I_Bdi,I_Gdi,I_Rdi

In [ ]:
def diffuse_new(blue,green,red,E2,E3,E4):
    E2_n=normalize(E2,256,0)
    E3_n=normalize(E3,256,0)
    E4_n=normalize(E4,256,0)
    I_Bd=blue.astype(int)
    I_Gd=green.astype(int)
    I_Rd=red.astype(int)
    E2_n=E2_n.astype(int)
    E3_n=E3_n.astype(int)
    E4_n=E4_n.astype(int)
    E2_n=E2_n.reshape(-1)
    E3_n=E3_n.reshape(-1)
    E4_n=E4_n.reshape(-1)
    I_Rd=I_Rd.reshape(-1)
    I_Gd=I_Gd.reshape(-1)
    I_Bd=I_Bd.reshape(-1)
    I_bpd = np.bitwise_xor(I_Bd, E4_n)
    I_gpd = np.bitwise_xor(I_Gd, E3_n)
    I_rpd = np.bitwise_xor(I_Rd, E2_n)
    I_rpd=np.reshape(I_rpd, (256, 256))
    I_gpd=np.reshape(I_gpd, (256, 256))
    I_bpd=np.reshape(I_bpd, (256, 256))
    E2_n=np.reshape(E2_n, (256, 256))
    E3_n=np.reshape(E3_n, (256, 256))
    E4_n=np.reshape(E4_n, (256, 256))
    I_Rd=np.reshape(I_Rd, (256, 256))
    I_Gd=np.reshape(I_Gd, (256, 256))
    I_Bd=np.reshape(I_Bd, (256, 256))
    I_Rdi=diffusion(I_rpd,I_Rd,I_Gd,E2_n)
    I_Gdi=diffusion(I_gpd,I_Gd,I_Bd,E3_n)
    I_Bdi=diffusion(I_bpd,I_Bd,I_Rd,E4_n)
    I_Rdi=I_Rdi.reshape(-1)
    I_Gdi=I_Gdi.reshape(-1)
    I_Bdi=I_Bdi.reshape(-1)
    return I_Bdi,I_Gdi,I_Rdi

In [ ]:
def mergeI(blue,green,red):
    Image = cv2.merge((blue,green,red))
    Image=np.array(Image)
    plt.imshow(Image)
    plt.show()
    return Image

In [ ]:
path=image_selector()
print(path)
print("Original Image")
OriginalImage = cv2.imread(path)
display(Image.open(path))
m,n,channel=OriginalImage.shape
print("pixels: {0}  width: {2} height: {1} ".format(m*n, m, n))
blue,green,red = split_into_rgb_channels(OriginalImage)
bluePer,greenPer,redPer = permute(blue,green,red,E1)
print("Image is permuted.....")
key=key_generation(E2)
blueEnc,greenEnc,redEnc,keyEnc=encode(bluePer,greenPer,redPer,key,E1,E2)
print("Image is encoded.....")
blueOpe,greenOpe,redOpe=operation(blueEnc,greenEnc,redEnc,keyEnc,E3)
print("DNA Operations are performed on Image.....")
blueDec,greenDec,redDec=decode(blueOpe,greenOpe,redOpe,E4)
print("Image is decoded.....")
blueDiff,greenDiff,redDiff=diffuse(blueDec,greenDec,redDec,E2,E3,E4)
print("Pixel Diffusion is performed on Image.....")
print("Encrypted Image")
EncImage=mergeI(blueDiff,greenDiff,redDiff)